# SQL_STE

## 1.1. Образовательные курсы. Оптимизация воронки

#### Дано три таблицы:  

1) peas:  
- st_id — ID ученика  
- timest — Время решения карточки  
- correct — равильно ли решена горошина?
- subject — Дисциплина, в которой находится горошина

2) studs:
- st_id — ID ученика  
- test_grp — Метка ученика в данном эксперименте  

3) final_project_check:
- st_id — ID ученика  
- sale_time — Время покупки  
- money — Цена, по которой приобрели данный курс
- subject — Дисциплина

#### Условие:

Образовательная платформа предлагает пройти студентам курсы по модели trial: студент может решить бесплатно лишь 30 горошин в день. Для неограниченного количества заданий в определенной дисциплине студенту необходимо приобрести полный доступ. Команда провела эксперимент, где был протестирован новый экран оплаты.

#### Задача:

Необходимо **в одном запросе** выгрузить следующую информацию о группах пользователей:

- ARPU 
- ARPAU 
- CR в покупку 
- СR активного пользователя в покупку 
- CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике
- ARPU считается относительно всех пользователей, попавших в группы.

**Активным** считается пользователь, за все время решивший **больше 10 задач** правильно в любых дисциплинах.  
**Активным** по математике считается пользователь, за все время решивший **2 или больше задач** правильно по математике.

#### Решение:

``` sql
WITH 
    ActiveAllSubject AS (
        SELECT st_id
        FROM  peas
        GROUP BY st_id
        HAVING SUM((correct)::int) > 10
    ), -- активные студенты
    ActiveMath AS (
        SELECT st_id
        FROM  peas
        WHERE subject = 'Math'
        GROUP BY st_id
        HAVING SUM((correct)::int) >= 2
    ), -- активные по математике
    Purchases AS (
        SELECT st_id, SUM(money) AS money
        FROM final_project_check
        GROUP BY st_id
    ), -- все студенты, купившие курсы + сумма покупок
    PurchasesMath AS (
        SELECT st_id
        FROM final_project_check
        WHERE subject = 'Math'
    ), -- студенты, купившие курс по математике
    Result AS (
        SELECT a.st_id, a.test_grp,
               CASE WHEN b.st_id IS NOT NULL THEN 1 ELSE 0 END AS active_flag,
               CASE WHEN c.st_id IS NOT NULL THEN 1 ELSE 0 END AS active_math_flag,
               CASE WHEN d.st_id IS NOT NULL THEN d.money ELSE 0 END AS money,
               CASE WHEN e.st_id IS NOT NULL THEN 1 ELSE 0 END AS purchases_math_flag
        FROM studs AS a LEFT JOIN ActiveAllSubject AS b ON a.st_id = b.st_id
                        LEFT JOIN ActiveMath AS c ON a.st_id = c.st_id
                        LEFT JOIN Purchases AS d ON a.st_id = d.st_id
                        LEFT JOIN PurchasesMath AS e ON a.st_id = e.st_id
    ) -- таблица с информацией по каждому студенту 
    
SELECT
    test_grp,
    SUM(money)::numeric / COUNT(st_id) AS arpu, -- ARPU = доход / все пользователи
    SUM(money)::numeric / NULLIF(COUNT(CASE WHEN active_flag = 1 THEN st_id END), 0) AS arpau, -- ARPAU = доход / активные пользователи
    COUNT(CASE WHEN money > 0 THEN st_id END)::numeric / COUNT(st_id) AS cr_purchase, -- CR в покупку 
    COUNT(CASE WHEN active_flag = 1 AND money > 0 THEN st_id END)::numeric 
        / NULLIF(COUNT(CASE WHEN active_flag = 1 THEN st_id END), 0) AS cr_active_purchase, -- CR активного пользователя в покупку
    COUNT(CASE WHEN active_math_flag = 1 AND purchases_math_flag = 1 THEN st_id END)::numeric
        / NULLIF(COUNT(CASE WHEN active_math_flag = 1 THEN st_id END), 0) AS cr_active_math_purchase_math  -- CR активного по математике в покупку курса по математике

FROM Result
GROUP BY test_grp
```

## 1.2. Очень усердные ученики

#### Условие:
Образовательные курсы состоят из различных уроков, каждый из которых состоит из нескольких маленьких заданий. Каждое такое маленькое задание называется "горошиной".

Назовём очень усердным учеником того пользователя, который хотя бы раз за текущий месяц правильно решил 20 горошин.

#### Задача:
Необходимо написать оптимальный запрос, который даст информацию о количестве очень усердных студентов.

#### Решение:

``` sql
WITH DiligentStudents AS (  
    SELECT st_id, COUNT(subject) AS count_tascks  
    FROM  peas  
    WHERE correct = true  
    GROUP BY st_id  
    HAVING COUNT(subject) >= 20  
)  

SELECT COUNT(st_id) AS diligent_students
FROM DiligentStudents;
```